## Statistical Learning and Deep Learning HW5

### Q1

In [1]:
import numpy as np
import pandas as pd
import glob
import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models

In [2]:
datasets = ['train', 'valid' ,'test']
labels = ['blazer', 'cardigan', 'coat', 'jacket']
base_path = '/tmp2/b06705028/sldl'
data_path = f'{base_path}/photos'

In [3]:
df = pd.DataFrame(columns=labels, index=datasets)
for ds in datasets:
    for lb in labels:
        basepath = os.path.join(f'{data_path}/{ds}/{lb}/', '*.jpg')
        cand_fn = glob.glob(basepath)
        df[lb][ds] = len(cand_fn)
df['total'] = df.sum(axis=1).astype('int')
print(df)

      blazer cardigan coat jacket  total
train     97      237  296    411   1041
valid      7       36   27     35    105
test       9       42   43     52    146


In [4]:
print('Ratio:')
df = df.drop(['total'], axis=1)
print (df.div(df.sum(axis=1), axis=0))

Ratio:
         blazer  cardigan      coat    jacket
train  0.093180  0.227666  0.284342  0.394813
valid  0.066667  0.342857  0.257143  0.333333
test   0.061644  0.287671  0.294521  0.356164


Given the number of instances of each image type, I suggest that the accuracy of the classification task will be jacket > coat > cardigan > blazer. This follows the hypothesis that larger number of instances in training set causes higher classification accuracy.

### Q2

In [5]:
# Image transformations
image_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.RandomResizedCrop(size=(224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=20),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    
    'valid':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
# Datasets from folders
data = {
    'train':
        ImageFolder(root=f'{data_path}/train/', transform=image_transforms['train']),
    'valid':
        ImageFolder(root=f'{data_path}/valid/', transform=image_transforms['valid']),
    'test':
        ImageFolder(root=f'{data_path}/test/', transform=image_transforms['test'])
}

In [7]:
# Dataloader
batch_size = 32
dataloaders = {
    'train':
        DataLoader(data['train'], batch_size=batch_size, shuffle=True, num_workers=10),
    'valid':
        DataLoader(data['valid'], batch_size=batch_size, shuffle=True, num_workers=10),
    'test':
        DataLoader(data['test'], batch_size=batch_size, shuffle=True, num_workers=10)
}

In [8]:
# device
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'using {device}')
print(torch.cuda.current_device())

using cuda
0


In [9]:
# loss
loss_fn = nn.CrossEntropyLoss()

In [10]:
def cross_entropy_loss(model, data_loader):
    loss = 0
    with torch.no_grad():
        for batch, (inputs, targets) in enumerate(data_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss += loss_fn(outputs, targets)
    return loss

In [11]:
def train(optim, model, weight_path, fix_weight=True, early_stop_patient=20, max_epoch=200):
    best_valid_loss = np.inf
    best_valid_epoch = 0
    for epoch in range(max_epoch):
        # train
        train_loss = 0
        for batch, (inputs, targets) in enumerate(dataloaders['train']):
            inputs, targets = inputs.to(device), targets.to(device)
            model.train()
            optim.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optim.step()
            train_loss += loss.item()
            
        # validation
        model.eval()
        valid_loss = cross_entropy_loss(model, dataloaders['valid']).cpu().numpy()
 
        # update weight if lower validation loss
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            best_valid_epoch = epoch
            torch.save(model, weight_path)
        # early stopping
        elif (epoch - best_valid_epoch >= early_stop_patient):
            print(f'early stopping at epoch {epoch}')
            return
        
        if (epoch % 10 == 0):
            print(f'epoch {epoch}: train_loss = {train_loss:.3f}, valid_loss = {valid_loss:.3f}, best_valid = {best_valid_loss:.3f}')
        

In [12]:
all_lr = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1]
for lr in all_lr:
    print('\n============================================')
    print(f'train SGD lr = {lr}')
    weight_path = f'{base_path}/Q2_weight_{lr}'
    
    # load pretrained resnet50 and set the output dimension to 4
    model = models.resnet50(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, 4)
    model.to(device)
    print(f'model at {device}')
    
    # train
    optim = torch.optim.SGD(model.parameters(), lr=lr)
    train(optim, model, weight_path)


train SGD lr = 0.0001
model at cuda
epoch 0: train_loss = 44.137, valid_loss = 5.291, best_valid = 5.291
epoch 10: train_loss = 42.456, valid_loss = 5.137, best_valid = 5.095
epoch 20: train_loss = 42.392, valid_loss = 5.001, best_valid = 4.957
epoch 30: train_loss = 41.907, valid_loss = 4.882, best_valid = 4.882
epoch 40: train_loss = 41.410, valid_loss = 4.913, best_valid = 4.870
epoch 50: train_loss = 41.360, valid_loss = 4.836, best_valid = 4.836
epoch 60: train_loss = 40.971, valid_loss = 4.916, best_valid = 4.770
epoch 70: train_loss = 40.790, valid_loss = 4.813, best_valid = 4.768
epoch 80: train_loss = 40.469, valid_loss = 4.840, best_valid = 4.730
epoch 90: train_loss = 40.226, valid_loss = 4.887, best_valid = 4.721
epoch 100: train_loss = 40.127, valid_loss = 4.795, best_valid = 4.559
epoch 110: train_loss = 39.941, valid_loss = 4.727, best_valid = 4.559
early stopping at epoch 119

train SGD lr = 0.0005
model at cuda
epoch 0: train_loss = 43.348, valid_loss = 5.135, best_va

#### SGD

In [14]:
for lr in all_lr:
    weight_path = f'{base_path}/Q2_weight_{lr}'
    print(f'loading {weight_path}')
    saved_model = torch.load(weight_path)
    test_size = len(data['test'])
    n_correct = 0
    with torch.no_grad():
        for batch, (inputs, targets) in enumerate(dataloaders['test']):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = saved_model(inputs)
            _, preds = torch.max(outputs, 1)
            correct = (targets==preds).cpu().numpy()
            n_correct += np.sum(correct)
    print(f'SGD, lr = {lr}: test accuracy: ', n_correct / len(data['test']))

loading /tmp2/b06705028/sldl/Q2_weight_0.0001
SGD, lr = 0.0001: test accuracy:  0.4452054794520548
loading /tmp2/b06705028/sldl/Q2_weight_0.0005
SGD, lr = 0.0005: test accuracy:  0.589041095890411
loading /tmp2/b06705028/sldl/Q2_weight_0.001
SGD, lr = 0.001: test accuracy:  0.5616438356164384
loading /tmp2/b06705028/sldl/Q2_weight_0.005
SGD, lr = 0.005: test accuracy:  0.636986301369863
loading /tmp2/b06705028/sldl/Q2_weight_0.01
SGD, lr = 0.01: test accuracy:  0.6301369863013698
loading /tmp2/b06705028/sldl/Q2_weight_0.1
SGD, lr = 0.1: test accuracy:  0.6027397260273972


### Adam

In [15]:
for lr in all_lr:
    print('\n============================================')
    print(f'train Adam lr = {lr}')
    weight_path = f'{base_path}/Q2_weight_adam_{lr}'
    
    # load pretrained resnet50 and set the output dimension to 4
    model = models.resnet50(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, 4)
    model.to(device)
    print(f'model at {device}')
    
    # train
    optim = torch.optim.SGD(model.parameters(), lr=lr)
    train(optim, model, weight_path)


train Adam lr = 0.0001
model at cuda
epoch 0: train_loss = 44.324, valid_loss = 5.541, best_valid = 5.541
epoch 10: train_loss = 42.422, valid_loss = 5.110, best_valid = 5.042
epoch 20: train_loss = 41.866, valid_loss = 5.101, best_valid = 4.947
epoch 30: train_loss = 41.413, valid_loss = 5.056, best_valid = 4.947
epoch 40: train_loss = 41.364, valid_loss = 5.018, best_valid = 4.908
epoch 50: train_loss = 40.940, valid_loss = 5.209, best_valid = 4.860
epoch 60: train_loss = 40.510, valid_loss = 5.050, best_valid = 4.848
epoch 70: train_loss = 40.604, valid_loss = 5.007, best_valid = 4.779
epoch 80: train_loss = 40.410, valid_loss = 5.018, best_valid = 4.678
epoch 90: train_loss = 40.117, valid_loss = 4.994, best_valid = 4.678
early stopping at epoch 92

train Adam lr = 0.0005
model at cuda
epoch 0: train_loss = 44.951, valid_loss = 5.433, best_valid = 5.433
epoch 10: train_loss = 41.549, valid_loss = 4.995, best_valid = 4.995
epoch 20: train_loss = 40.325, valid_loss = 4.714, best_val

In [16]:
for lr in all_lr:
    weight_path = f'{base_path}/Q2_weight_adam_{lr}'
    print(f'loading {weight_path}')
    saved_model = torch.load(weight_path)
    test_size = len(data['test'])
    n_correct = 0
    with torch.no_grad():
        for batch, (inputs, targets) in enumerate(dataloaders['test']):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = saved_model(inputs)
            _, preds = torch.max(outputs, 1)
            correct = (targets==preds).cpu().numpy()
            n_correct += np.sum(correct)
    print(f'SGD, lr = {lr}: valid accuracy: ', n_correct / len(data['test']))

loading /tmp2/b06705028/sldl/Q2_weight_adam_0.0001
SGD, lr = 0.0001: valid accuracy:  0.4041095890410959
loading /tmp2/b06705028/sldl/Q2_weight_adam_0.0005
SGD, lr = 0.0005: valid accuracy:  0.589041095890411
loading /tmp2/b06705028/sldl/Q2_weight_adam_0.001
SGD, lr = 0.001: valid accuracy:  0.5753424657534246
loading /tmp2/b06705028/sldl/Q2_weight_adam_0.005
SGD, lr = 0.005: valid accuracy:  0.6164383561643836
loading /tmp2/b06705028/sldl/Q2_weight_adam_0.01
SGD, lr = 0.01: valid accuracy:  0.6301369863013698
loading /tmp2/b06705028/sldl/Q2_weight_adam_0.1
SGD, lr = 0.1: valid accuracy:  0.5753424657534246
